In [1]:
import parse, textfields
import spacy
import numpy as np
nlp = spacy.load('en')

In [12]:
trump_par_texts = parse.read_paragraphs('nss/trump_nss.txt')
obama_par_texts = parse.read_paragraphs('nss/obama_nss.txt')
par_texts = trump_par_texts + obama_par_texts
k = len(trump_par_texts)
len(par_texts), len(trump_par_texts), len(obama_par_texts)

(550, 400, 150)

In [19]:
tokenized_pars = [parse.parse_doc(par) for par in nlp.pipe(par_texts)]
BOW, vocab, zerosel = parse.make_bow_matrix(tokenized_pars, min_df=2, min_doc_wordcount=5)
zeroind = np.argwhere(zerosel)[:,0]
par_texts = [pt for i,pt in enumerate(par_texts) if i not in zeroind]
tokenized_pars = [pt for i,pt in enumerate(tokenized_pars) if i not in zeroind]
BOW.shape, len(par_texts), len(tokenized_pars)

((443, 2279), 443, 443)

In [20]:
print(vocab[:5])
BOW[:5,:5]

["'s", '(', ')', ',', '-']


array([[0, 0, 0, 6, 0],
       [0, 0, 0, 3, 0],
       [0, 0, 0, 6, 1],
       [0, 0, 0, 8, 1],
       [0, 0, 0, 4, 0]])

In [21]:
PPMI = textfields.calc_ppmi(BOW)
SVD = textfields.calc_svd(PPMI, 10)
SVD = SVD - SVD.mean(axis=0) # center
SVD = SVD / np.linalg.norm(SVD, axis=1)[:,np.newaxis] # normalize
PPMI.shape, SVD.shape

((443, 2279), (443, 10))

In [22]:
dists = SVD.dot(SVD.T).sum(axis=1)
par_lens = np.array([len(toks) for toks in tokenized_pars])
print('correlation of word freq with distance:', np.corrcoef(par_lens, dists)[0,1])
print('norm of mean:', np.linalg.norm(SVD.mean(axis=0)))

correlation of word freq with distance: -0.7825176951170127
norm of mean: 0.10193117676032244
